# Proto-historic navigation performance data: exploratory work
> Author: Matteo Tomasini

In this script I simply explore a few concepts to attempt to explore data. This notebook helps me also test snippets of code that are then transferred to the `processing.py` and `plotting.py` files. 

In [1]:
import pandas as pd